In [2]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    import dateutil.parser
    import os
    from tqdm import tqdm
    %matplotlib inline

In [3]:
root= '/datasets_1/sagarj/BellLabs/CitySV_Data/city_centers_sampling/'

In [4]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [5]:
mapillary_Image_ID = 'https://a.mapillary.com/v3/images?client_id=bkJxVGFaQkdEcVY0RnVseWduREE0aTowZWMzMmY1Y2M3MjQ4ZTgx&closeto='
distanceFrag = '&radius=150'
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [6]:
%store -r gdf

In [7]:
gdf.head(n=2)

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance,centroid
0,"POLYGON ((-0.51038 51.46809, -0.51036 51.46795...",51.691874,51.286760,0.334016,-0.510375,256874343,relation,65606,51.507322,-0.127647,"London, Greater London, England, United Kingdom",boundary,administrative,0.940783,POINT (-0.10941 51.50050)
1,"POLYGON ((-87.94010 42.00093, -87.94003 41.998...",42.023040,41.644531,-87.524081,-87.940101,309792632,relation,122604,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States",boundary,administrative,0.961530,POINT (-87.68569 41.83806)


In [8]:
cities = os.listdir(root)

In [ ]:
for c in tqdm(cities):
    cityRoot = root +'/'+ c
    city = c.split(',')[0].strip()
    data = gpd.read_file(cityRoot+'/shapeFiles/samples.shp')
    print("Total Sample points ",len(data) , c)
    count = 1
    for point in tqdm(data['geometry']):
        imagesMeta = []
        name= city + str(count) + '.json'
        loc= str(point.x)+','+str(point.y)
    #     print(loc)
        finalUrl = mapillary_Image_ID + loc + distanceFrag
        try:
            response = rq.get(finalUrl,timeout=10)
            dictionary = json.loads(response.text)
            imagesMeta = dictionary["features"]
            logger.debug('Working with %d image at location %s'%(count,loc))
        except:
            logger.error('Request Timed out')
    #     print(imagesMeta)
        logger.debug('Found %d Images in this file'%(count))
    #     print(imagesMeta)
        json.dump(imagesMeta,open(cityRoot+ '/' + 'pointsJson/' + name,'w'))
        count+=1
    print(os.listdir(root +'/'+ c))

  0%|          | 0/3464 [00:00<?, ?it/s]

Total Sample points  3464 Helsinki, Helsinki sub-region, Uusimaa, Southern Finland, Mainland Finland, Finland



  4%|▍         | 148/3464 [09:30<4:02:41,  4.39s/it]


  8%|▊         | 294/3464 [18:45<2:27:52,  2.80s/it]


 13%|█▎        | 440/3464 [27:54<2:04:38,  2.47s/it]


...

 42%|████▏     | 887/2099 [41:41<15:32,  1.30it/s]


 49%|████▉     | 1035/2099 [48:04<2:08:09,  7.23s/it]


 56%|█████▋    | 1181/2099 [54:31<1:42:13,  6.68s/it]


 63%|██████▎   | 1326/2099 [1:01:34<43:58,  3.41s/it]  


 65%|██████▌   | 1369/2099 [1:03:29<22:12,  1.83s/it]